# **1. 기본 전처리**

## RandomForest

### 1) Panda전략 : Babysitting one model

* max_depth=120, min_samples_leaf=10, min_samples_split=15,n_estimators=500, random_state=42
* R2 : 0.70689
* RMSE : 281.27173962231535

### 2) Caviar전략 : Training many models in parallel

* RandomizedSearchCV

In [ ]:
paras = {'n_estimators':[100,120,140,160,180,200],
         'max_depth': [50,60,70,80,90,100], 
         'min_samples_split':[5,10,20,30,40,50],
         'min_samples_leaf':[15,20,25,30,35,40]}

In [ ]:
model = RandomForestRegressor(random_state=42)
model3 = RandomizedSearchCV(model,paras,verbose = 2, cv=3, random_state = 42)
model3.fit(train_x, train_y.values.ravel())

In [ ]:
model3.best_params_

{'n_estimators': 180,
 'min_samples_split': 50,
 'min_samples_leaf': 15,
 'max_depth': 70}

In [ ]:
# n_estimators = 180, min_samples_split = 50, min_samples_leaf = 15, max_depth = 70
# R2 : 0.70772
# RMSE : 280.8726099030807

## Lasso

In [ ]:
lasso = Lasso(alpha=0.0001, max_iter=100000)
lasso.fit(train_x,train_y)
pred = lasso.predict(test_x)

In [ ]:
print("훈련 세트의 정확도 : {:.2f}".format(lasso.score(train_x, train_y)))
print("테스트 세트의 정확도 : {:.2f}".format(lasso.score(test_x, test_y)))
print("사용한 특성의 수 : {}".format(np.sum(lasso.coef_ != 0)))
print("사용한 max_iter : {}".format(lasso.n_iter_))

훈련 세트의 정확도 : 0.73
테스트 세트의 정확도 : 0.71
사용한 특성의 수 : 105
사용한 max_iter : 42846


In [ ]:
print(mean_squared_error(test_y, pred, squared = False))
print(r2_score(test_y, pred))

279.3870660291991
0.710806437805957


## Ridge

In [ ]:
ridge = Ridge().fit(train_x,train_y)
ridge01 = Ridge(alpha=0.1).fit(train_x,train_y)
ridge10 = Ridge(alpha=10).fit(train_x,train_y)
ridge_pred = ridge.predict(test_x)
ridge_pred01 = ridge01.predict(test_x)
pridge_red10 = ridge10.predict(test_x)

In [ ]:
print("훈련 세트의 정확도 : {:.2f}".format(ridge.score(train_x, train_y)))
print("테스트 세트의 정확도 : {:.2f}".format(ridge.score(test_x, test_y)))
print("사용한 특성의 수 : {}".format(np.sum(ridge.coef_ != 0)))
print("사용한 max_iter : {}".format(ridge.n_iter_))

훈련 세트의 정확도 : 0.73
테스트 세트의 정확도 : 0.71
사용한 특성의 수 : 108
사용한 max_iter : None


In [ ]:
print("훈련 세트의 정확도 : {:.2f}".format(ridge01.score(train_x, train_y)))
print("테스트 세트의 정확도 : {:.2f}".format(ridge01.score(test_x, test_y)))
print("사용한 특성의 수 : {}".format(np.sum(ridge01.coef_ != 0)))
print("사용한 max_iter : {}".format(ridge01.n_iter_))

훈련 세트의 정확도 : 0.73
테스트 세트의 정확도 : 0.71
사용한 특성의 수 : 108
사용한 max_iter : None


In [ ]:
print("훈련 세트의 정확도 : {:.2f}".format(ridge10.score(train_x, train_y)))
print("테스트 세트의 정확도 : {:.2f}".format(ridge10.score(test_x, test_y)))
print("사용한 특성의 수 : {}".format(np.sum(ridge10.coef_ != 0)))
print("사용한 max_iter : {}".format(ridge10.n_iter_))

훈련 세트의 정확도 : 0.73
테스트 세트의 정확도 : 0.71
사용한 특성의 수 : 108
사용한 max_iter : None


# **2.새로운 전처리 전략**

## 1) 날씨 데이터 추가

[기본 데이터]
* 학습데이터 : onenavi_train.csv(7월 20일에서 24일 사이의 수도권 3~15km 주행데이터)
* 평가데이터 : onenavi_evaluation.csv(7월 27일에서 31일 사이의 수도권 3~15km 주행데이터)

[추가 데이터]
* 주소(시군구)데이터 : onenavi_pnu.csv(주행데이터를 기준으로 출발지의 주소 정보, key : RID)
* 신호등(갯수)데이터 : onenavi_signal.csv(주행데이터를 기준으로 경로의 신호등 갯수, key : RID)
* 날씨데이터 : onenavi_weather.csv(주행데이터를 기준으로 해당 일자의 수도권 날씨 정보, key : RID)

[변수 소개]
* RID : RouteID(경로 아이디로 주행 시 생성)
* TIME_DEPARTUREDATE : 출발시간
* TIME_ARRIVEDATE : 도착시간
* A_DISTANCE : 실 주행 거리
* ET : 실제 주행 시간
* ETA : 원내비 예측 주행 시간
* ETAA : 예측 주행 시간 정확도


1. pnu 합치기: level1_pnu와 level2_pnu 간의 변수 종속성이 의심됨

In [ ]:
data['pnu'] = data['level1_pnu'] + ' ' + data['level2_pnu']
data.drop(columns=['level1_pnu', 'level2_pnu'], inplace=True)

2. prefix를 같은 값을 준다면 변수명이 겹치는 일이 일어남 -> 영향을 줄지는 모르겠지만 일단 변경

In [ ]:
data = pd.get_dummies(data, columns=['WEEKDAY'], prefix='weekday', drop_first=True)
data = pd.get_dummies(data, columns=['HOUR'], prefix='hour', drop_first=True)
data = pd.get_dummies(data, columns=['pnu'], prefix='pnu', drop_first=True)

## 2) 도시 교통 체증 데이터 feature 추가

In [ ]:
raw_data_2_train_count

,level1_pnu,level2_pnu,HOUR,DAY,RID
0,경기도,가평군,1,22,1
1,경기도,가평군,3,23,1
2,경기도,가평군,6,22,1
3,경기도,가평군,6,23,1
4,경기도,가평군,8,20,1
...,...,...,...,...,...
6554,인천광역시,연수구,9,23,1
6555,인천광역시,연수구,17,21,1
6556,인천광역시,중구,9,23,1
6557,인천광역시,중구,10,24,1


In [ ]:
raw_data_2_train = pd.merge(raw_data_2_train, raw_data_2_train_count, on = ['level1_pnu', 'level2_pnu','HOUR','DAY'])
raw_data_2_train

,RID_x,A_DISTANCE,ET,ETA,ETAA,level1_pnu,level2_pnu,signaltype,PerHour,WEEKDAY,HOUR,DAY,RID_y
0,router-84875df7fc-b5nxc-7-20255166-0,12914.0,1201.534058,987.624634,82.196974,경기도,광명시,7.0,38.692536,0,5,20,2
1,router-84875df7fc-8n4sw-6-20478618-0,3133.0,244.360001,283.873749,83.829699,경기도,광명시,3.0,46.156490,0,5,20,2
2,router-84875df7fc-b5nxc-6-10888379-0,7483.0,823.817017,855.933716,96.101477,서울특별시,영등포구,31.0,32.699980,0,5,20,5
3,router-84875df7fc-b5nxc-1-6273195-0,4404.0,288.588013,298.466125,96.577088,서울특별시,영등포구,2.0,54.937833,0,5,20,5
4,router-84875df7fc-b5nxc-1-6273884-0,5277.0,546.008972,648.721497,81.188491,서울특별시,영등포구,10.0,34.792835,0,5,20,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55428,router-84875df7fc-cmkz9-7-56731250-0,10130.0,735.458008,745.946228,98.573920,경기도,용인시 처인구,15.0,49.585428,4,23,24,2
55429,router-84875df7fc-8n4sw-2-7744879-0,5029.0,551.458008,390.452301,70.803632,경기도,포천시,19.0,32.830061,4,23,24,1
55430,router-84875df7fc-b5nxc-4-60270096-0,10619.0,899.088013,808.205933,89.891748,경기도,구리시,28.0,42.519085,4,23,24,1
55431,router-84875df7fc-b5nxc-7-25052984-0,10306.0,785.568970,830.402222,94.292894,경기도,오산시,15.0,47.228953,4,23,24,1


# **3.사용 모델**

* xgb
* catboost
* 딥러닝
* ensemble stacking
* BayesianRidge



## 1) xgb

* 1) 데이터 처리

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

# train_x = train_x.drop(columns=['temp', 'precipitation'])
# test_x = test_x.drop(columns=['temp', 'precipitation'])

xgb_model = XGBRegressor(n_estimators=100,
                     gamma=1,
                     eta=0.1,
                     max_depth=5,
                     reg_lambda=5,
                     reg_alpha=5,
                     random_state=42
                    )
xgb_model.fit(train_x, train_y.values.ravel())
pred = xgb_model.predict(test_x)
print(round(r2_score(test_y, pred), 5)) # 0.72319
print(round(mean_squared_error(test_y, pred, squared=False), 5)) # 212.13837

* 2) 데이터 처리

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

train_y = train_y['ET']
test_y = test_y['ET']

xgb_model = XGBRegressor(n_estimators=100,
                     gamma=1,
                     eta=0.1,
                     max_depth=5,
                     reg_lambda=5,
                     reg_alpha=5,
                     random_state=42
                    )
xgb_model.fit(train_x, train_y.values.ravel())
pred = xgb_model.predict(test_x)
print(round(r2_score(test_y, pred), 5)) # 0.73264
print(round(mean_squared_error(test_y, pred, squared=False), 5)) # 254.63692

## 2) catboost

* 0) 데이터 처리 x

In [ ]:
from catboost import CatBoostRegressor

cat = CatBoostRegressor()

In [ ]:
cat.fit(train_x,train_y)

In [ ]:
test_y['ET']

0        555.546997
1        397.209015
2        720.645996
3        943.086975
4        196.244003
            ...    
53737    438.523010
53738    458.688995
53739    478.945007
53740    223.481995
53741    330.881989
Name: ET, Length: 53742, dtype: float64

In [ ]:
pred = cat.predict(test_x)
r2_score(test_y['ET'], pred)

0.7378173293909649

* 1) 데이터 처리 

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
pred = cat2.predict(test_x_with_weather)
r2_score(test_y_with_weather, pred)

0.7549756853440808

* 2) 데이터 처리 

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
print(r2_score(test_y['ET'],pred))

0.7345078827317261


### 튜닝

In [ ]:
# 아래에 실습코드를 작성하세요.
from catboost import CatBoostRegressor

cat = CatBoostRegressor()

# 최적의 파라미터 값 찾아보기
from sklearn.model_selection import GridSearchCV
# CAT
param_cat2 = {"depth" : [6,8,10],
          "iterations" : [250,500,700],
          "learning_rate" : [0.1,0.2,0.3], 
          "l2_leaf_reg" : [2,10,20]
          }

# cv = StratifiedKFold(n_splits=5, shuffle = True, random_state=42)
gscv_cat2 = GridSearchCV (estimator = cat, param_grid = param_cat2, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
gscv_cat2.fit(train_x_with_weather,train_y_with_weather)

* 1) 데이터 처리

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
pred2 = gscv_cat2.predict(test_x_with_weather)
r2_score(test_y_with_weather, pred2)

0.7430666820995188

* 2) 데이터 처리

In [ ]:
pred = cat.predict(test_x)
r2_score(test_y['ET'], pred)

0.7378173293909649

## 3) 딥러닝 cnn

In [ ]:
keras.backend.clear_session()

model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(104,)))
model.add(keras.layers.Dense(400, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(200, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(24, activation='relu'))
model.add(keras.layers.Dense(12, activation='relu'))
model.add(keras.layers.Dense(6, activation='relu'))
model.add(keras.layers.Dense(1))

model.compile(loss='mse', optimizer='adam', metrics=[keras.metrics.mae])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 400)               42000     
                                                                 
 dropout (Dropout)           (None, 400)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               80200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               20100     
                                                                 
 dense_3 (Dense)             (None, 50)                5050      
                                                                 
 dense_4 (Dense)             (None, 24)                1

In [ ]:
keras.backend.clear_session()

model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(104,)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(24, activation='relu'))
model.add(keras.layers.Dense(12, activation='relu'))
model.add(keras.layers.Dense(6, activation='relu'))
model.add(keras.layers.Dense(1))

model.compile(loss='mse', optimizer='adam', metrics=[keras.metrics.mae])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               10500     
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 24)                1224      
                                                                 
 dense_3 (Dense)             (None, 12)                300       
                                                                 
 dense_4 (Dense)             (None, 6)                 78        
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 17,159
Trainable params: 17,159
Non-traina

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', min_delta=0, patience=7, restore_best_weights=True)

In [ ]:
model.fit(x_train, y_train, epochs=50, verbose=1, validation_split=0.2, callbacks=[es])

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
pred = model.predict(x_test)
print(r2_score(y_test, pred))

1677/1677 [==============================] - 2s 1ms/step
0.7411101333949084


## 4) ensemble stacking

In [ ]:
from catboost import CatBoostRegressor
cat_final = CatBoostRegressor()

 

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

 

# 개별모델 내부에서 CV 적용해 Stacking하는 함수 구현
def get_stacking_datasets(model, x_train_n, y_train_n, x_test_n, n_folds):
    # CV하기 위해 K-fold 설정
    kf = KFold(n_splits=n_folds, shuffle=False)
    
    # 최종 메타 모델이 사용할 학습 데이터 반환을 위해서 넘파이 배열을 0으로 만들어서 초기화
    train_fold_pred = np.zeros((x_train_n.shape[0], 1)) # 2차원으로
    test_pred = np.zeros((x_test_n.shape[0], n_folds))
    print(model.__class__.__name__, '모델 시작')
    
    for folder_counter, (train_idx, valid_idx) in enumerate(kf.split(x_train_n)):
        # 개별 모델 내부에서 학습하고 1개의 fold로 예측할 데이터 셋 추출
        print(f" Fold 횟수 : {folder_counter+1}")
        x_tr = x_train_n.iloc[train_idx]
        y_tr = y_train_n.iloc[train_idx]
        x_te = x_train_n.iloc[valid_idx]
        
        # 개별 모델이 학습한 후 1개의 fold데이터셋으로 예측값 반환 후 최종 메타모델이 학습할 데이터셋에 첨가
        model.fit(x_tr, y_tr)
        train_fold_pred[valid_idx, :] = model.predict(x_te).reshape(-1,1)
        # 개별 모델이 원본 데이터셋의 검증 데이터셋을 기반으로 예측 결과값 반환 후 최종 메타모델이 검증할 데이터셋에 첨가
        test_pred[:,folder_counter] = model.predict(x_test_n).reshape(-1)
    
    # 개별모델안에서 테스트 데이터셋을 기반으로 예측한 결과값들 mean취해주고 2차원으로 바꾸어주기
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    return train_fold_pred, test_pred_mean

 

lr_train, lr_test = get_stacking_datasets(lr, train_x, train_y, test_x, 5)
rf_train, rf_test = get_stacking_datasets(rf, train_x, train_y, test_x, 5)
gb_train, gb_test = get_stacking_datasets(gb, train_x, train_y, test_x, 5)
xgb_train, xgb_test = get_stacking_datasets(xgb, train_x, train_y, test_x, 5)

In [ ]:
stack_final_x_train = np.concatenate((lr_train, rf_train, gb_train, xgb_train), axis=1)
stack_final_x_test = np.concatenate((lr_test, rf_test, gb_test, xgb_test), axis=1)

 

# 최종 메타모델로 학습
# 최종 메타모델 학습시 label은 원본 데이터의 label(y값)
xgb.fit(stack_final_x_train, train_y)
stack_final_pred = xgb.predict(stack_final_x_test)

## 5) BayesianRidge

* 2) 데이터 처리

In [ ]:
from sklearn.linear_model import BayesianRidge
model_BayesianRidge=BayesianRidge()
model_BayesianRidge.fit(train_x,train_y)

BayesianRidge()

In [ ]:
pred_BayesianRidge=model_BayesianRidge.predict(test_x)
print(r2_score(test_y,pred_BayesianRidge))

0.7256817327597592
